In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Tue Dec 13 18:11:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   36C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 40%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/otto_ii/xgtrain_25.csv')
test_x = dd.read_csv('../input/otto_ii/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-12-13 18:11:56,085 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-13 18:11:56,085 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-12-13 18:11:56,100 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-13 18:11:56,100 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'num_class':9,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    lloss = log_loss(test_y, predictions)
    
    return lloss

In [12]:
study = optuna.create_study(direction='minimize')

[I 2022-12-13 18:12:06,998] A new study created in memory with name: no-name-021a43cc-5146-4a55-bdf1-3febb0b29d8d


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:12:07] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[18:12:07] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1
[I 2022-12-13 18:48:37,008] Trial 0 finished with value: 0.4713420497885604 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.03327518666086722, 'alpha': 9.068874482224441, 'colsample_bytree': 0.6510658245083794, 'subsample': 0.43399667886063187, 'learning_rate': 0.047013949161952555, 'max_depth': 24, 'min_child_weight': 232, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.4713420497885604.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:48:37] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new ran

CPU times: user 1min 6s, sys: 29.5 s, total: 1min 36s
Wall time: 37min 50s


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:49:58] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[18:49:58] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1
[I 2022-12-13 20:01:17,736] Trial 2 finished with value: 0.49754043412714766 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.034567770800385915, 'alpha': 0.2700533393149023, 'colsample_bytree': 0.9969684815307955, 'subsample': 0.5453014925978478, 'learning_rate': 0.06448768786961717, 'max_depth': 25, 'min_child_weight': 171, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.4713420497885604.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[20:01:17] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new ra

CPU times: user 27min 6s, sys: 12min 56s, total: 40min 3s
Wall time: 14h 47min 21s


In [15]:
study.best_trial.params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 1.561807471397023,
 'alpha': 0.008369504276601328,
 'colsample_bytree': 0.4992096678150335,
 'subsample': 0.5108551101398487,
 'learning_rate': 0.018416095567506383,
 'max_depth': 22,
 'min_child_weight': 72,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/otto_ii/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})


predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[09:37:20] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[09:37:20] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


0.4405786118857353

In [20]:
%%time
params = study.best_trial.params
params['seed'] = 2022
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[09:50:23] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[09:50:23] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


CPU times: user 26.5 s, sys: 10.2 s, total: 36.7 s
Wall time: 13min 8s


0.44173575817005506

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['num_class']= 9
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    
    error = log_loss(real_test_y, predictions)

    scores.append(error)
    print(error)

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:03:32] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[10:03:32] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


0.44117495728142464
1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:16:40] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[10:16:40] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


0.4412326568806053
2


...

0.44133347285957036
13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[12:53:15] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[12:53:15] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


0.44106340192851745
14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[13:06:15] task [xgboost.dask-0]:tcp://127.0.0.1:42677 got new rank 0
[13:06:15] task [xgboost.dask-1]:tcp://127.0.0.1:41053 got new rank 1


0.4408284928245249
CPU times: user 5min 49s, sys: 2min 58s, total: 8min 47s
Wall time: 3h 15min 47s


In [22]:
scores

[0.44117495728142464,
 0.4412326568806053,
 0.4413808885517996,
 0.4412168447307903,
 0.4407866148642117,
 0.4420600246573842,
 0.44092503294617724,
 0.4419707299545841,
 0.4421486023302065,
 0.44198355075083945,
 0.44130346069624077,
 0.44053758430148204,
 0.44133347285957036,
 0.44106340192851745,
 0.4408284928245249]

In [23]:
import numpy as np
np.mean(scores)

0.44132975437055727

/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
/home/tunguz/.local/